In [76]:
from py2neo import Graph
import requests

In [77]:
# Connect to Neo4j instance
graph = Graph("http://52.2.207.222/db/data")

In [184]:
# CYPHER CREATE STATEMENTS

MOVIE_CONSTRAINT = '''
    CREATE CONSTRAINT ON (f:Film) ASSERT f.url IS UNIQUE
'''

CREATE_MOVIE_QUERY = '''
MERGE (f:Film {url: {url}})
SET f.created = {created},
    f.edited = {edited},
    f.episode_id = toInt({episode_id}),
    f.opening_crawl = {opening_crawl},
    f.release_date = {release_date},
    f.title = {title}
WITH f
UNWIND split({director}, ",") AS director
MERGE (d:Director {name: director})
CREATE UNIQUE (f)-[:DIRECTED_BY]->(d)
WITH f
UNWIND split({producer}, ",") AS producer
MERGE (p:Producer {name: producer})
CREATE UNIQUE (f)-[:PRODUCED_BY]->(p)
WITH f
UNWIND {characters} AS character
MERGE (c:Person {url: character})
CREATE UNIQUE (c)-[:APPEARS_IN]->(f)
WITH f
UNWIND {planets} AS planet
MERGE (p:Planet {url: planet})
CREATE UNIQUE (f)-[:TAKES_PLACE_ON]->(p)
WITH f
UNWIND {species} AS specie
MERGE (s:Species {url: specie})
CREATE UNIQUE (s)-[:APPEARS_IN]->(f)
WITH f
UNWIND {starships} AS starship
MERGE (s:Starship {url: starship})
CREATE UNIQUE (s)-[:APPEARS_IN]->(f)
WITH f
UNWIND {vehicles} AS vehicle
MERGE (v:Vehicle {url: vehicle})
CREATE UNIQUE (v)-[:APPEARS_IN]->(f)
'''

FIND_NEW_PERSON_QUERY = '''
MATCH (p:Person) WHERE size(keys(p)) < 2 
WITH rand() as r, p ORDER BY r LIMIT 1
WITH p
RETURN p.url AS url LIMIT 1
'''

FIND_NEW_PLANET_QUERY = '''
MATCH (p:Planet) WHERE size(keys(p)) < 2 
WITH rand() as r, p ORDER BY r LIMIT 1
WITH p
RETURN p.url AS url LIMIT 1
'''

FIND_NEW_SPECIES_QUERY = '''
MATCH (s:Species) WHERE size(keys(s)) < 2
WITH rand() AS r, s ORDER By r LIMIT 1
WITH s
RETURN s.url AS url LIMIT 1
'''

FIND_NEW_VEHICLE_QUERY = '''
MATCH (s:Vehicle) WHERE size(keys(s)) < 2
WITH rand() AS r, s ORDER By r LIMIT 1
WITH s
RETURN s.url AS url LIMIT 1
'''

FIND_NEW_STARSHIP_QUERY = '''
MATCH (s:Starship) WHERE size(keys(s)) < 2
WITH rand() AS r, s ORDER By r LIMIT 1
WITH s
RETURN s.url AS url LIMIT 1
'''

CREATE_PERSON_QUERY = '''
MERGE (p:Person {url: {url}})
SET p.birth_year = {birth_year},
    p.created = {created},
    p.edited = {edited},
    p.eye_color = {eye_color},
    p.gender = {gender},
    p.hair_color = {hair_color},
    p.height = {height},
    p.mass = {mass},
    p.name = {name},
    p.skin_color = {skin_color}
WITH p
MERGE (home:Planet {url: {homeworld}})
CREATE UNIQUE (home)<-[:IS_FROM]-(p)
WITH p
UNWIND {species} AS specie
MERGE (s:Species {url: specie})
CREATE UNIQUE (p)-[:IS_SPECIES]->(s)
WITH p
UNWIND {starships} AS starship
MERGE (s:Starship {url: starship})
CREATE UNIQUE (p)-[:PILOTS]->(s)
WITH p
UNWIND {vehicles} AS vehicle
MERGE (v:Vehicle {url: vehicle})
CREATE UNIQUE (p)-[:PILOTS]->(v)
'''

CREATE_PLANET_QUERY = '''
MERGE (p:Planet {url: {url}})
SET p.created = {created},
    p.diameter = {diameter},
    p.edited = {edited},
    p.gravity = {gravity},
    p.name = {name},
    p.orbital_period = {orbital_period},
    p.population = {population},
    p.rotation_period = {rotation_period},
    p.surface_water = {surface_water}
WITH p
UNWIND split({climate}, ",") AS c
MERGE (cli:Climate {type: c})
CREATE UNIQUE (p)-[:HAS_CLIMATE]->(cli)
WITH p
UNWIND split({terrain}, ",") AS t
MERGE (ter:Terrain {type: t})
CREATE UNIQUE (p)-[:HAS_TERRAIN]->(ter)
'''

CREATE_SPECIES_QUERY = '''
MERGE (s:Species {url: {url}})
SET s.name = {name},
    s.language = {language},
    s.average_height = {average_height},
    s.average_lifespan = {average_lifespan},
    s.classification = {classification},
    s.created = {created},
    s.designation = {designation},
    s.eye_colors = {eye_colors},
    s.hair_colors = {hair_colors},
    s.skin_colors = {skin_colors}
'''

CREATE_VEHICLE_QUERY = '''
MERGE (v:Vehicle {url: {url}})
SET v.cargo_capacity = {cargo_capacity},
    v.consumables = {consumables},
    v.cost_in_credits = {cost_in_credits},
    v.created = {created},
    v.crew = {crew},
    v.edited = {edited},
    v.length = {length},
    v.max_atmosphering_speed = {max_atmosphering_speed},
    v.model = {model},
    v.name = {name},
    v.passengers = {passengers}
MERGE (m:Manufacturer {name: {manufacturer}})
CREATE UNIQUE (v)-[:MANUFACTURED_BY]->(m)
WITH v
MERGE (c:VehicleClass {type: {vehicle_class}})
CREATE UNIQUE (v)-[:IS_CLASS]->(c)
'''

CREATE_STARSHIP_QUERY = '''
MERGE (s:Starship {url: {url}})
SET s.MGLT = {MGLT},
    s.consumables = {consumables},
    s.cost_in_credits = {cost_in_credits},
    s.created = {created},
    s.crew = {crew},
    s.edited = {edited},
    s.hyperdrive_rating = {hyperdrive_rating},
    s.length = {length},
    s.max_atmosphering_speed = {max_atmosphering_speed},
    s.model = {model},
    s.name = {name},
    s.passengers = {passengers}
MERGE (m:Manufacturer {name: {manufacturer}})
CREATE UNIQUE (s)-[:MANUFACTURED_BY]->(m)
WITH s
MERGE (c:StarshipClass {type: {starship_class}})
CREATE UNIQUE (s)-[:IS_CLASS]->(c)
'''

In [79]:
# Exploring the API
r = requests.get("http://swapi.co/api/")
r.json()

{'films': 'http://swapi.co/api/films/',
 'people': 'http://swapi.co/api/people/',
 'planets': 'http://swapi.co/api/planets/',
 'species': 'http://swapi.co/api/species/',
 'starships': 'http://swapi.co/api/starships/',
 'vehicles': 'http://swapi.co/api/vehicles/'}

In [191]:
for i in range(1,7):
    url = "http://swapi.co/api/films/" + str(i) + "/"
    #print(url)
    r = requests.get(url)
    params = r.json()
    #print(params)
    graph.cypher.execute(CREATE_MOVIE_QUERY, params)

In [186]:
while True:
    
    result = graph.cypher.execute(FIND_NEW_PERSON_QUERY)
    #print(result)
    #url = result.one
    #print(url)
    #r = requests.get(url)
    #params = r.json()
    #graph.cypher.execute(CREATE_PERSON_QUERY, params)

   | url                           
---+--------------------------------
 1 | http://swapi.co/api/people/78/

http://swapi.co/api/people/78/
   | url                           
---+--------------------------------
 1 | http://swapi.co/api/people/29/

http://swapi.co/api/people/29/
   | url                           
---+--------------------------------
 1 | http://swapi.co/api/people/72/

http://swapi.co/api/people/72/
   | url                           
---+--------------------------------
 1 | http://swapi.co/api/people/56/

http://swapi.co/api/people/56/
   | url                          
---+-------------------------------
 1 | http://swapi.co/api/people/7/

http://swapi.co/api/people/7/
   | url                           
---+--------------------------------
 1 | http://swapi.co/api/people/50/

http://swapi.co/api/people/50/
   | url                           
---+--------------------------------
 1 | http://swapi.co/api/people/59/

http://swapi.co/api/people/59/
   | url         

TypeError: a bytes-like object is required, not 'str'

In [187]:
while True:
    result = graph.cypher.execute(FIND_NEW_PLANET_QUERY)
    url = result.one
    r = requests.get(url)
    params = r.json()
    graph.cypher.execute(CREATE_PLANET_QUERY, params)

TypeError: a bytes-like object is required, not 'str'

In [188]:
while True:
    result = graph.cypher.execute(FIND_NEW_SPECIES_QUERY)
    url = result.one
    r = requests.get(url)
    params = r.json()
    graph.cypher.execute(CREATE_SPECIES_QUERY, params)

TypeError: a bytes-like object is required, not 'str'

In [189]:
while True:
    result = graph.cypher.execute(FIND_NEW_VEHICLE_QUERY)
    url = result.one
    r = requests.get(url)
    params = r.json()
    graph.cypher.execute(CREATE_VEHICLE_QUERY, params)


TypeError: a bytes-like object is required, not 'str'

In [190]:
while True:
    result = graph.cypher.execute(FIND_NEW_STARSHIP_QUERY)
    url = result.one
    r = requests.get(url)
    params = r.json()
    graph.cypher.execute(CREATE_STARSHIP_QUERY, params)


TypeError: a bytes-like object is required, not 'str'